In [46]:
from datetime import date
import pandas as pd

file = '../../tmp/fact_health_activity_base.csv'
date_fields = ['creation_date', 'start_date', 'end_date']
df = pd.read_csv(file, parse_dates=date_fields)
df['time_min'] = (df['end_date'] - df['start_date']).dt.total_seconds() / 60
df.head()

,type,source_name,unit,creation_date,start_date,end_date,value,time_min
0,HKQuantityTypeIdentifierHeartRate,Dimithree’s Apple Watch,count/min,2022-11-14 04:07:44-07:00,2022-11-13 22:39:20-07:00,2022-11-13 22:39:20-07:00,70.0000,0.0
1,HKQuantityTypeIdentifierHeartRate,Dimithree’s Apple Watch,count/min,2022-11-14 04:13:08-07:00,2022-11-14 04:11:47-07:00,2022-11-14 04:11:47-07:00,59.4325,0.0
2,HKQuantityTypeIdentifierHeartRate,Dimithree’s Apple Watch,count/min,2022-11-14 04:13:50-07:00,2022-11-14 04:11:16-07:00,2022-11-14 04:11:16-07:00,61.0000,0.0
3,HKQuantityTypeIdentifierHeartRate,Dimithree’s Apple Watch,count/min,2022-11-14 04:18:55-07:00,2022-11-14 04:14:08-07:00,2022-11-14 04:14:08-07:00,60.0000,0.0
4,HKQuantityTypeIdentifierHeartRate,Dimithree’s Apple Watch,count/min,2022-11-14 04:23:48-07:00,2022-11-14 04:19:58-07:00,2022-11-14 04:19:58-07:00,69.0000,0.0


In [63]:
# TODO: calculate the difference between start times in seconds and put in own column
df = df.sort_values(['type', 'source_name', 'start_date'])
df['next_start_seconds'] = (df['start_date'] - df['start_date'].shift(1)).dt.total_seconds()
df.head()

,type,source_name,unit,creation_date,start_date,end_date,value,time_min,next_start_seconds
727804,HKQuantityTypeIdentifierBasalEnergyBurned,Dimithree’s Apple Watch,Cal,2022-11-13 19:45:05-07:00,2022-11-13 19:29:05-07:00,2022-11-13 19:43:53-07:00,17.482,14.800000,NaN
727805,HKQuantityTypeIdentifierBasalEnergyBurned,Dimithree’s Apple Watch,Cal,2022-11-13 19:59:54-07:00,2022-11-13 19:43:53-07:00,2022-11-13 19:58:44-07:00,15.961,14.850000,888.0
727806,HKQuantityTypeIdentifierBasalEnergyBurned,Dimithree’s Apple Watch,Cal,2022-11-13 20:15:15-07:00,2022-11-13 19:58:44-07:00,2022-11-13 20:13:34-07:00,17.337,14.833333,891.0
727807,HKQuantityTypeIdentifierBasalEnergyBurned,Dimithree’s Apple Watch,Cal,2022-11-13 20:29:35-07:00,2022-11-13 20:13:34-07:00,2022-11-13 20:28:25-07:00,15.966,14.850000,890.0
727808,HKQuantityTypeIdentifierBasalEnergyBurned,Dimithree’s Apple Watch,Cal,2022-11-13 20:44:36-07:00,2022-11-13 20:28:35-07:00,2022-11-13 20:43:25-07:00,14.407,14.833333,901.0


In [65]:
# TODO: look at seconds between start times and see what average values are by activity
df.groupby(['type', 'source_name', 'start_date'])['next_start_seconds'].mean()

AttributeError: 'str' object has no attribute 'dt'

In [47]:
activity_list = df['type'].unique().tolist()
activity_list

['HKQuantityTypeIdentifierHeartRate',
 'HKQuantityTypeIdentifierStepCount',
 'HKQuantityTypeIdentifierDistanceWalkingRunning',
 'HKQuantityTypeIdentifierBasalEnergyBurned',
 'HKQuantityTypeIdentifierDistanceCycling']

In [48]:
walk_run = df[df['type']=='HKQuantityTypeIdentifierDistanceWalkingRunning']
daily_walk_detail = pd.DataFrame(walk_run.groupby([walk_run['start_date'].dt.date, 'source_name'])['value'].sum()).reset_index()
daily_walk_detail[daily_walk_detail['start_date'] == date(2024, 8, 20)]

,start_date,source_name,value
3608,2024-08-20,Dimithree’s Apple Watch,9.630648
3609,2024-08-20,Dimithree’s iphone,8.793305


In [49]:
# TODO: take the min start, max start values for both watch and iphone and see if there is overlap
daily_grouped = walk_run.groupby([walk_run['start_date'].dt.date, 'source_name'])
daily_grouped['start_date'].min()

start_date  source_name            
2016-04-12  Dimithree’s iphone        2016-04-12 17:18:05-07:00
2016-04-13  Dimithree’s iphone        2016-04-13 06:27:46-07:00
2016-04-14  Dimithree’s iphone        2016-04-14 00:19:59-07:00
2016-04-15  Dimithree’s iphone        2016-04-15 07:09:53-07:00
2016-04-16  Dimithree’s iphone        2016-04-16 14:27:08-07:00
                                                 ...           
2024-08-23  Dimithree’s Apple Watch   2024-08-23 00:46:49-07:00
            Dimithree’s iphone        2024-08-23 06:04:10-07:00
2024-08-24  Dimithree’s Apple Watch   2024-08-24 02:08:03-07:00
            Dimithree’s iphone        2024-08-24 04:11:07-07:00
2024-08-25  Dimithree’s Apple Watch   2024-08-25 06:21:37-07:00
Name: start_date, Length: 3619, dtype: datetime64[ns, UTC-07:00]

In [62]:
test = pd.DataFrame()
test['start_min'] = daily_grouped['start_date'].min()
test['end_date'] = daily_grouped['end_date'].max()
test['time_minutes'] = daily_grouped['time_min'].sum()
test['miles'] = daily_grouped['value'].sum()
test = test.reset_index()
test.head()

TypeError: cannot subtract DatetimeArray from SeriesGroupBy

In [54]:
test[test['start_date'] == date(2024, 8, 20)]

,start_date,source_name,start_min,end_date,time_minutes,miles
3608,2024-08-20,Dimithree’s Apple Watch,2024-08-20 00:14:08-07:00,2024-08-20 23:53:00-07:00,225.65,9.630648
3609,2024-08-20,Dimithree’s iphone,2024-08-20 05:20:49-07:00,2024-08-20 18:36:00-07:00,194.40,8.793305


In [55]:
df[df['start_date'] == date(2024, 8, 20)]

,type,source_name,unit,creation_date,start_date,end_date,value,time_min
